# SQL in Jupyter Notebooks
### Prerequisites
**Installing Python packages**
```
pip install ipython-sql
pip install pyodbc
```
[ipython-sql documentation](https://pypi.org/project/ipython-sql/)

**SQL driver installation**
[ODBC Driver 18 for SQL Server](https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16)
(Tested on MS SQL Server 2022)

### Connection to SQL Server
**%sql** for single-line queries.
**%%sql** for multi-line queries.
The **<<** operator captures query results in a local variable.

In [3]:
# Loading magic method sql
%load_ext sql

# Connection
user = 'SA'
password = 'dockersql#1' #'YourPassword'
server = 'localhost'
port = '1433'
database = 'master'
driver = 'odbc+driver+18+for+sql+server'

connection_string = f'mssql+pyodbc://{user}:{password}@{server}:{port}/{database}?encrypt=no&driver={driver}'

%sql $connection_string

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
(pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]Client unable to establish connection because an error was encountered during handshakes before login. Common causes include client attempting to connect to an unsupported version of SQL Server, server too busy to accept new connections or a resource limitation (memory or maximum allowed connections) on the server. (26) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Istniejące połączenie zostało gwałtownie zamknięte przez zdalnego hosta.\r\n (10054); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Client unable to establish connection (26); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Client unable to establish connection due to prelogin failure (10054)')
(Background on this error at: https://sqlalche.me/e/14/e3q8)
Connection info needed in SQLAlchemy format, example:
               p

In [1]:
# Test connection
%sql SELECT * FROM sys.databases

: Msg 102, Level 15, State 1, Line 1
Incorrect syntax near 'connection'.

Total execution time: 00:00:00.015

# Usługa uczenia maszynowego w Microsoft SQL Server

Usługa uczenia maszynowego w Microsoft SQL Server to funkcja programu, która umożliwia uruchamianie skryptów w językach Python i R bezpośrednio z bazy danych. Dzięki temu, możliwe jest wykorzystanie różnych narzędzi i bibliotek do przeprowadzania analiz predykcyjnych i uczenia maszynowego na danych zgromadzonych w bazie bez konieczności przenoszenia ich poza SQL Server. Usługa ta jest przydatna do automatyzacji i usprawnienia procesów związanych z uczeniem maszynowym.

Standardowo obraz dockera Microsoft SQL Server __nie posiada__ serwisu SQL Machine Learning, aby stworzyć taki kontener trzeba posłużyć się tym przykładem [dockerfile'a](https://github.com/Microsoft/mssql-docker/tree/master/linux/preview/examples/mssql-mlservices).

# Skrypty Python w SQL Server

[**Opis języka Python**](https://docs.python.org/3/tutorial/index.html)

Python jest zainstalowany niezależnie w systemie Microsoft SQL Server w ramach SQL Machine Learning.

Skrypt w języku Python można uruchomić przez przekazanie go jako argument do procedury składowanej [sp\_execute\_external\_script](https://learn.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-execute-external-script-transact-sql?view=sql-server-ver16). Ta procedura systemowa pozwala na uruchomienie środowiska Pythona w kontekście uczenia maszynowego SQL, przesyłanie danych do Pythona, zarządzanie sesjami użytkownika Pythona i zwracanie wyników do klienta.

## Zezwolenie na działanie procedury sp\_execute\_external\_script

In [1]:
sp_configure 'external scripts enabled', 1;
RECONFIGURE;

Configuration option 'external scripts enabled' changed from 0 to 1. Run the RECONFIGURE statement to install.

Total execution time: 00:00:00.081

## Przykłady użycia skryptów Python w SQL

### Parametry procedury sp\_execute\_external\_script:

**@language** – język. Przyjmowane wartości to R, Python i język zdefiniowany za pomocą [CREATE EXTERNAL LANGUAGE](https://learn.microsoft.com/en-us/sql/t-sql/statements/create-external-language-transact-sql?view=sql-server-ver16). \
**@script** – treść skryptu. Cały skrypt musi być zapisany jako tekst Unicode. Można również przekazać tekst przez zmienną typu nvarchar.

In [6]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'

print("Hello world!")

'

STDOUT message(s) from external script: 
Hello world!

Total execution time: 00:00:03.064

In [1]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
ls = [1, 2.0, 3, "abc"]

for element in ls:
    print(element)
'

STDOUT message(s) from external script: 
1
2.0
3
abc

Total execution time: 00:00:03.889

Uczenie maszynowe w systemie Microsoft SQL Server w zakresie obsługi języka Python opiera się na pakiecie pandas. Podstawowy obiekt, poprzez który przekazywane są dane do i z Pythona to ramka danych (DataFrame), dlatego dane wejściowe muszą być tabelaryczne, np. utworzone zdaniem SELECT. Dane wyjściowe również muszą być przedstawione w postaci tego typu obiektu.

**@input\_data\_1 = N'input\_data\_1'** - Dane wejściowe wykorzystywane przez zewnętrzny skrypt w postaci zapytania Transact-SQL.  
**@input\_data\_1\_name = N'input\_data\_1\_name'** - Nazwa zmiennej używanej do reprezentowania zapytania zdefiniowanego przez @input\_data\_1. Domyślną wartością jest **InputDataSet**.

**@output\_data\_1\_name = N'output\_data\_1\_name'** - Nazwa zmiennej w zewnętrznym skrypcie, która zawiera dane zwracane do serwera SQL po zakończeniu wywołania procedury składowanej. Domyślną wartością jest **OutputDataSet**.

In [14]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT 1, N''ABC'''


(1 row affected)

Total execution time: 00:00:02.925

(No column name),(No column name)
1,ABC


In [22]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
import pandas as pd
import matplotlib.pyplot as plt

if isinstance(InputDataSet, pd.DataFrame):
    print("InputDataSet is a DataFrame")
'
    , @input_data_1 = N'SELECT 1, N''ABC'''

STDOUT message(s) from external script: 
InputDataSet is a DataFrame

Total execution time: 00:00:03.733

Nazywanie kolumn wynikowych:

In [1]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT 1, N''ABC'''
WITH RESULT SETS(([Number] INT, Text NVARCHAR(10)));
GO


(1 row affected)

Total execution time: 00:00:03.329

Number,Text
1,ABC


In [2]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT ProductID, ProductName, CategoryID FROM Northwind.dbo.Products'
WITH RESULT SETS((ProductID INT, ProductName NVARCHAR(40), CategoryID INT));
GO


(77 rows affected)

Total execution time: 00:00:02.697

ProductID,ProductName,CategoryID
1,Chai,1
2,Chang,1
3,Aniseed Syrup,2
4,Chef Anton's Cajun Seasoning,2
5,Chef Anton's Gumbo Mix,2
6,Grandma's Boysenberry Spread,2
7,Uncle Bob's Organic Dried Pears,7
8,Northwoods Cranberry Sauce,2
9,Mishi Kobe Niku,6
10,Ikura,8


Pewne typy danych nie są wspierane, należy wówczas wykorzystać funkcję CAST, warto też zamieniać na właściwsze typy w wynikach za pomocą WITH RESULTS SETS.

In [3]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT ProductID, ProductName, UnitPrice FROM Northwind.dbo.Products;'

: Msg 39004, Level 16, State 20, Line 1
A 'Python' script error occurred during execution of 'sp_execute_external_script' with HRESULT 0x80004004.

: Msg 39019, Level 16, State 2, Line 1
An external script error occurred: 

Unsupported input data type in column 'UnitPrice'.  Supported types: bit, tinyint, smallint, int, bigint, uniqueidentifier, real, float, char, varchar, nchar, nvarchar, varbinary, date, datetime, smalldatetime.

Total execution time: 00:00:07.740

In [7]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT ProductID, ProductName, CAST(UnitPrice AS FLOAT) 
	FROM Northwind.dbo.Products; '
WITH RESULT SETS((ProductID INT, ProductName NVARCHAR(10), UnitPrice MONEY));
GO

(77 rows affected)

Total execution time: 00:00:02.753

ProductID,ProductName,UnitPrice
1,Chai,"18,00"
2,Chang,"19,00"
3,Aniseed Sy,"10,00"
4,Chef Anton,"22,00"
5,Chef Anton,"21,35"
6,Grandma's,"25,00"
7,Uncle Bob',"30,00"
8,Northwoods,"40,00"
9,Mishi Kobe,"97,00"
10,Ikura,"31,00"
